In [2]:
import pandas as pd
import numpy as np

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# demo = "../data/raw/df_final_demo.txt"
# webdata1 = "../data/raw/df_final_web_data_pt_1.txt"
# webdata2 = "../data/raw/df_final_web_data_pt_2.txt"
# experiment = "../data/raw/df_final_experiment_clients.txt"

# df_demo = pd.read_csv(demo)
# df_webdata1 = pd.read_csv(webdata1)
# df_webdata2 = pd.read_csv(webdata2)
# df_exp = pd.read_csv(experiment)

In [3]:
df_demo = pd.read_csv('/Users/sashacrowe/codebase-ih/vanguard-ab-test/data/raw/df_final_demo.txt')

## First Look at the Dataset

In [4]:
df_demo.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0


In [5]:
df_demo.shape

(70609, 9)

In [7]:
df_demo.dtypes

client_id             int64
clnt_tenure_yr      float64
clnt_tenure_mnth    float64
clnt_age            float64
gendr                object
num_accts           float64
bal                 float64
calls_6_mnth        float64
logons_6_mnth       float64
dtype: object

**DATA BREAKDOWN**    
Column names are fine -> nothing to change here:
- client_id (int64) -> unique number series
- clnt_tenure_yr (float64) -> how long the client has been tenured in yrs
- clnt_tenure_mnth (float64) -> how long the client has been tenured in mnths **exactly, so more than yrs!!!**
- clnt_age (float64) -> age of client -> **....WHY is it float? There are half years**
- gendr (object: string) -> gender of client, categorial more than 2 options it seems
- num_accts (float64) -> the number of accounts the client has
- bal (float64) -> client balance
- **calls_6_mnth (float64) -> not sure exactly**
- logons_6_mnth (float64) -> how many times the client has logged on in the last 6 months?

In [8]:
# Checking for missing data
df_demo.isnull().sum().sort_values(ascending=False)

clnt_age            15
clnt_tenure_yr      14
clnt_tenure_mnth    14
gendr               14
num_accts           14
bal                 14
calls_6_mnth        14
logons_6_mnth       14
client_id            0
dtype: int64

In [15]:
df_demo['gendr'].unique()

array(['U', 'M', 'F', nan, 'X'], dtype=object)

In [11]:
# checking out the null rows

null_rows = df_demo[df_demo.isnull().any(axis=1)]
print(null_rows)

       client_id  clnt_tenure_yr  clnt_tenure_mnth  clnt_age gendr  num_accts  \
4164     7402828             NaN               NaN       NaN   NaN        NaN   
8316      355337             NaN               NaN       NaN   NaN        NaN   
8677     8412164             NaN               NaN       NaN   NaN        NaN   
9583     4666211             8.0             106.0       NaN     F        2.0   
13444    2222915             NaN               NaN       NaN   NaN        NaN   
18066    4876926             NaN               NaN       NaN   NaN        NaN   
25961    5277910             NaN               NaN       NaN   NaN        NaN   
28432    7616759             NaN               NaN       NaN   NaN        NaN   
35323    8191345             NaN               NaN       NaN   NaN        NaN   
43518    1227228             NaN               NaN       NaN   NaN        NaN   
46076    8611797             NaN               NaN       NaN   NaN        NaN   
47189    5144725            

In [12]:
#percent null
(14/df_demo.shape[0])*100

0.01982750074353128

In [ ]:
# drop it like it's hot
#clean_ /df_demo = df_demo.dropna()


In [14]:
df_demo.describe().round(2)

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,num_accts,bal,calls_6_mnth,logons_6_mnth
count,70609.00,70595.00,70595.00,70594.00,70595.00,70595.00,70595.00,70595.00
mean,5004991.88,12.05,150.66,46.44,2.26,147445.24,3.38,5.57
std,2877277.63,6.87,82.09,15.59,0.53,301508.71,2.24,2.35
min,169.00,2.00,33.00,13.50,1.00,13789.42,0.00,1.00
25%,2519329.00,6.00,82.00,32.50,2.00,37346.84,1.00,4.00
50%,5016978.00,11.00,136.00,47.00,2.00,63332.90,3.00,5.00
75%,7483085.00,16.00,192.00,59.00,2.00,137544.90,6.00,7.00
max,9999839.00,62.00,749.00,96.00,8.00,16320040.15,7.00,9.00


## What needs cleaning
- there are 14 clients we can drop cause their entire col rows are blank (affects less than 1% of the dataset)
- only one client is missing an age, but we can fill that with avg age
- clnt_age -> **suggest to round down cause half years are meaningless**
- num_accts, clnt_tenure_yr, logons_6_mnth & calls_6_mnth should be full integers (cause discrete values) -> **suggest to round these up**
- gendr has U & X as well as M & F -> **suggest to leave alone**
- **we should not drop anything until we merge/join the tables together**